<a href="https://colab.research.google.com/github/harithaharry001/CovidDataset/blob/master/resnet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [9]:
from os import listdir
from matplotlib import image
from skimage.transform import resize

def load_data(path):
    
    train_files = []
    for foldername1 in listdir(path):
        filepath1 = path  + "/" + foldername1
        for filename1 in listdir(filepath1):
            train_files.append(filepath1 + "/" + filename1)
            
    # Original Dimensions
    image_width = 224
    image_height = 224
    channels = 3
    
    loaded_images = np.ndarray(shape=(len(train_files), image_height, image_width, channels),dtype=np.float32)
    print(type(loaded_images))
    loaded_class = []
    i = 0
    for foldername in listdir(path):
        filepath = path  + "/" + foldername
        print("Folder : ",filepath)
        for filename in listdir(filepath):
            # load image
            img_data = image.imread(filepath + "/" + filename)
            
            # store loaded image
            img_data = resize(img_data, (224, 224, 3))
            loaded_images[i] = img_data
            loaded_class.append(foldername)
            i = i + 1
            #print('> loaded %s %s' % (filename, img_data.shape))
        print('Loaded: ',i , ' images from ',filepath)
            
    return loaded_images,loaded_class


In [ ]:
path = r"/content/drive/MyDrive/Dataset"
x,y = load_data(path)
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

<class 'numpy.ndarray'>
Folder :  /content/drive/MyDrive/Dataset/Covid
Loaded:  478  images from  /content/drive/MyDrive/Dataset/Covid
Folder :  /content/drive/MyDrive/Dataset/Normal
Loaded:  2358  images from  /content/drive/MyDrive/Dataset/Normal


In [ ]:
print(train_x.shape)
print(test_x.shape)

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:
def identity_block(X, f, filters):

    # Retrieve Filters
    F1, F2, F3 = filters  # F1=64,F2=64,256
    
    X_shortcut = X
    
    # First  layer
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    
    # Second  layer
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third  layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    # Final step: Add shortcut value to F(X), and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
def convolutional_block(X, f, filters, s = 2):

    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First layer 
    X = Conv2D(F1, (1, 1), strides = (s,s))(X) # 1,1 is filter size
    X = BatchNormalization(axis = 3)(X)  # normalization on channels
    X = Activation('relu')(X)

      
    # Second layer  (f,f)=3*3 filter by default
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)


    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)


    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X


In [ ]:
def ResNet50(input_shape=(224, 224, 3), classes=2):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """

    # Define the input with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) #3,3 padding

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X) 
    X = BatchNormalization(axis=3)(X) 
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
   
    # below 3 lines are the conv layers from convolutional_block function defined above
    #X = Conv2D(F1, (1, 1), strides = (s,s))(X)
    #X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
   
    X = identity_block(X, 3, [64, 64, 256]) 
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
  
    X = identity_block(X, 3, [64, 64, 256])
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)


    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (224, 224, 3), classes = 2)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_265 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
batch_normalization_265 (BatchN (None, 112, 112, 64) 256         conv2d_265[0][0]                 
___________________________________________________________________________________________

In [ ]:
import pandas as pd
from keras.utils import to_categorical
y_train_values, unique = pd.factorize(train_y)
print('y_train ', y_train_values, unique)


y_test_values, unique = pd.factorize(test_y)
print('y_test ', y_test_values, unique)

y_train_one_hot = to_categorical(y_train_values)
y_test_one_hot = to_categorical(y_test_values)


y_train  [0 0 0 ... 1 1 1] ['Normal' 'Covid']
y_test  [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0
 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

In [ ]:
x_train_normalization = train_x / 255.0
x_test_normalization = test_x / 255.0

In [ ]:
from sklearn.utils import shuffle


x_shuffled_default, y_shuffled_default = shuffle(x_train_normalization, y_train_one_hot)

In [20]:
model.fit(x_train_normalization, y_train_one_hot, epochs = 30, batch_size = 32)

Epoch 1/30
52/52 [==============================] - 1097s 21s/step - loss: 2.9679 - accuracy: 0.7774
Epoch 2/30
52/52 [==============================] - 1079s 21s/step - loss: 0.3243 - accuracy: 0.8305
Epoch 3/30
52/52 [==============================] - 1085s 21s/step - loss: 0.2711 - accuracy: 0.8633
Epoch 4/30
52/52 [==============================] - 1073s 21s/step - loss: 0.3023 - accuracy: 0.8880
Epoch 5/30
52/52 [==============================] - 1081s 21s/step - loss: 0.4789 - accuracy: 0.8478
Epoch 6/30
52/52 [==============================] - 1073s 21s/step - loss: 0.3269 - accuracy: 0.8625
Epoch 7/30
52/52 [==============================] - 1075s 21s/step - loss: 0.1966 - accuracy: 0.9181
Epoch 8/30
52/52 [==============================] - 1079s 21s/step - loss: 0.1760 - accuracy: 0.9149
Epoch 9/30
52/52 [==============================] - 1075s 21s/step - loss: 0.1746 - accuracy: 0.9319
Epoch 10/30
52/52 [==============================] - 1073s 21s/step - loss: 0.1433 - accura

In [7]:
preds = model.evaluate(x_test_normalization, y_test_one_hot)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

NameError: ignored

In [3]:
from matplotlib.pyplot import imread
from keras.preprocessing import image
img_path = '/content/drive/MyDrive/NORMAL(1576).jpg'
img = image.load_img(img_path, target_size=(224, 224,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = imread(img_path)
imshow(my_image)
print(model.predict(x))

FileNotFoundError: ignored